In [ ]:
!pip install spacy

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

# Importing Libraries

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy

## Custom faq (generated with the help of Deepsek)

In [ ]:
faqs = {
    # General Questions
    "What are your opening hours?": "We are open from 9:00 AM to 8:00 PM, Monday to Saturday. Closed on Sundays.",
    "Where are you located?": "Our headquarters is at 123 Business Park, Cityville, USA. You can find store locations on our website.",
    "How can I contact customer support?": "Call us at +1-800-123-4567 or email support@company.com. Live chat is also available on our website.",
    "Do you have a physical store?": "Yes, we have multiple stores across the country. Check our website for the nearest location.",
    "Is your website secure for payments?": "Absolutely! We use SSL encryption and trusted payment gateways like PayPal and Stripe.",

    # Delivery & Shipping
    "Do you offer delivery?": "Yes, we deliver nationwide. International shipping is available for select products.",
    "What are your delivery charges?": "Delivery fees vary based on location and order size. Free shipping is available for orders over $50.",
    "How long does delivery take?": "Standard delivery takes 3-5 business days. Express options (1-2 days) are available at extra cost.",
    "Do you ship internationally?": "Yes, we ship to over 50 countries. Shipping costs and times depend on the destination.",
    "Can I track my order?": "Yes, you’ll receive a tracking number via email once your order is dispatched.",

    # Returns & Refunds
    "Can I return a product?": "Yes, we accept returns within 30 days of purchase, provided the item is unused and in original packaging.",
    "What is your return policy?": "Items must be returned within 30 days with a receipt. Refunds are processed within 5-7 business days.",
    "How do I initiate a return?": "Log into your account and go to 'Order History' to start the return process, or contact our support team.",
    "Do you offer refunds or exchanges?": "We offer both! Refunds go back to your original payment method, or you can request an exchange.",
    "Who pays for return shipping?": "Customers cover return shipping unless the return is due to our error (e.g., wrong item delivered).",

    # Products & Inventory
    "Do you sell mobile phones?": "Yes, we offer the latest smartphones from brands like Apple, Samsung, and Google.",
    "Are your products brand new?": "All products are 100% new and come with a manufacturer’s warranty unless stated otherwise.",
    "Do you sell refurbished items?": "Yes, we have a certified refurbished section with discounted prices and warranty coverage.",
    "How do I check product availability?": "Visit the product page on our website for real-time stock updates, or call your nearest store.",
    "Do you offer gift wrapping?": "Yes! Gift wrapping is available at checkout for a small fee.",

    # Payments & Discounts
    "What payment methods do you accept?": "We accept credit/debit cards (Visa, MasterCard, Amex), PayPal, Apple Pay, and bank transfers.",
    "Do you offer installment plans?": "Yes, select products qualify for 0% EMI options through our financing partners.",
    "How can I apply a discount code?": "Enter the code at checkout under the 'Promo Code' field and click 'Apply'.",
    "Do you have a loyalty program?": "Yes! Join our rewards program to earn points on every purchase, redeemable for discounts.",
    "Are taxes included in the price?": "Prices displayed on the website exclude taxes. Taxes are calculated at checkout based on your location.",

    # Technical Support
    "How do I reset my account password?": "Click 'Forgot Password' on the login page and follow the instructions sent to your email.",
    "Why is my account locked?": "Accounts are locked after multiple failed login attempts. Contact support to unlock it.",
    "How do I update my personal details?": "Log into your account, go to 'Profile Settings,' and edit your information.",
    "Do you have a mobile app?": "Yes! Download our app from the App Store or Google Play for exclusive deals and easier shopping.",
    "Is my data safe with you?": "We adhere to GDPR and other privacy laws. Your data is never shared with third parties without consent.",

    # Warranty & Repairs
    "Do products come with a warranty?": "Most products include a 1-year manufacturer’s warranty. Extended warranties are available for purchase.",
    "How do I claim warranty service?": "Contact our support team with your order number and issue details to initiate a warranty claim.",
    "Do you offer repair services?": "Yes, we partner with authorized service centers for repairs. Visit our 'Support' page for details.",
    "What’s covered under warranty?": "Warranty covers manufacturing defects but not accidental damage or misuse. Check terms for specifics.",
    "How long do repairs take?": "Repairs typically take 7-14 days, depending on parts availability and the issue.",

    # Corporate & Bulk Orders
    "Do you offer corporate discounts?": "Yes, we provide special pricing for bulk orders. Email sales@company.com for a quote.",
    "Can I order custom products?": "Custom orders are possible for certain categories (e.g., branded merchandise). Contact our sales team.",
    "Do you work with resellers?": "We have a reseller program. Apply on our website or email partnerships@company.com.",
    "What’s your minimum order quantity (MOQ)?": "MOQ varies by product. Discuss your needs with our sales team for details.",
    "Do you provide invoices for businesses?": "Yes, VAT/GST-compliant invoices are generated automatically for all orders.",

    # Miscellaneous
    "Can I cancel my order?": "Orders can be canceled within 24 hours of placement if they haven’t been shipped yet.",
    "Do you donate to charities?": "We support various NGOs through our CSR program. Learn more on our 'Social Responsibility' page.",
    "How do I leave a product review?": "Log into your account, go to 'Order History,' and click 'Write a Review' for the purchased item.",
    "Do you have a newsletter?": "Yes! Subscribe on our website for updates on promotions, new arrivals, and exclusive offers.",
    "How do I unsubscribe from emails?": "Click 'Unsubscribe' at the bottom of any marketing email, or update preferences in your account.",
}

# Text Preprocessing

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
corpus=[]
for key in faqs.keys():
    review=re.sub('[^a-zA-Z]',' ', key)
    review=review.lower()
    # review=word_tokenize(review)
    doc = nlp(review)
    review=[token.lemma_ for token in doc if token.text.lower() not in stop_words]
    review=' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['opening hour',
 'locate',
 'contact customer support',
 'physical store',
 'website secure payment',
 'offer delivery',
 'delivery charge',
 'long delivery take',
 'ship internationally',
 'track order',
 'return product',
 'return policy',
 'initiate return',
 'offer refund exchange',
 'pay return shipping',
 'sell mobile phone',
 'product brand new',
 'sell refurbish item',
 'check product availability',
 'offer gift wrapping',
 'payment method accept',
 'offer installment plan',
 'apply discount code',
 'loyalty program',
 'taxis include price',
 'reset account password',
 'account lock',
 'update personal detail',
 'mobile app',
 'datum safe',
 'product come warranty',
 'claim warranty service',
 'offer repair service',
 'cover warranty',
 'long repair take',
 'offer corporate discount',
 'order custom product',
 'work reseller',
 'minimum order quantity   moq  ',
 'provide invoice business',
 'cancel order',
 'donate charity',
 'leave product review',
 'newsletter',
 'unsubscrib

In [ ]:
len(corpus)

45

# Initializing Tf-Idf

In [ ]:
tv=TfidfVectorizer(max_features=1000,ngram_range=(1,3)) # Mono, bi and trigram have been used
X=tv.fit_transform(corpus).toarray()

In [ ]:
X.shape

(45, 180)

# Preprocessing Input Text & Generating Output

In [97]:
def check_faq(faq):
  review=re.sub('[^a-zA-z]',' ', faq)
  review=review.lower()
  doc = nlp(review)
  review=[token.lemma_ for token in doc if token.text.lower() not in stop_words]
  review= ' '.join(review)

  q = tv.transform([review])
  similarities = cosine_similarity(q, X)
  # Cosine similarity measures the closeness of each sentence with the given input sentence. Uncomment the below line to check the cosine similarity

  # print(similarities)

  threshold = np.max(similarities)
  print(f"Maximum Cosine Similarity: {threshold}")

  # Argmax returns the index of the most similar vector
  if threshold > 0.5:
    most_similar_idx = np.argmax(similarities)
    for i, (key,val) in enumerate(faqs.items()):
      if i == most_similar_idx:
        return key
  else:
    return f'Sorry, I couldn’t find a suitable answer to your question.'

# Testing the chatbot

In [ ]:
ques = "will i get a warranty?"

In [ ]:
print(check_faq(ques))

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.36577451 0.35472637 0.         0.50724665 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.        ]]
What’s covered under warranty?


In [98]:
print(check_faq("Do you have secure payment method?"))

Maximum Cosine Similarity: 0.531862310285909
Is your website secure for payments?


In [99]:
print(check_faq("Is there any discounts??"))

Maximum Cosine Similarity: 0.39180522752718433
Sorry, I couldn’t find a suitable answer to your question.


# Welcome to XYZ Company's Chatbot

#### Run the cell below to have a conversation with the chatbot

In [ ]:
question = input("Enter your question: ")
faqs[check_faq(question)]

Enter your question: My account is locked. what to do??


'Accounts are locked after multiple failed login attempts. Contact support to unlock it.'

In [ ]:
question = input("Enter your question: ")
faqs[check_faq(question)]

In [96]:
np.max([1,2,43,4,5])

np.int64(43)